# Relatório TP Parte II: Aprendizagem Automática 2
## Microsoft Malware Prediction
 
<br>
<br>

- Carlos Gonçalves    a77278
-  José Ferreira    a78452
-  Ricardo Peixoto    a78587

#### Mestrado Integrando em Engenharia Informática, Universidade do Minho

# Introdução

Nesta fase do trabalho o objetivo passava pela utilização de métodos de *machine learning* considerados *deep*, em oposição aos métodos mais tradicionais utilizados na primeira fase, comparando a performance entre os diferentes modelos. Como este trabalho é uma continuidade da fase anterior existem certos tópicos que já foram tratados e não serão abordados novamente. Fazem parte desses assuntos a análise e tratamento dos dados recolhidos. 
Apesar disso o grupo sentiu que, com esta nova abordagem, era possível a utilização de uma maior quantidade de atributos. Assim sendo decidimos que, além do *dataset* utilizado na primeira fase, iriamos também extender esse *dataset* e utilizar adicionalmente algumas variáveis que foram desconsideradas anteriormente. 

## Variáveis reconsideradas

![alt](img.png)

Como podemos ver na imagem em cima apresentada, estas foram as variáveis eliminadas na primeira fase pelo excesso de atributos que continham.Isso deve-se ao facto de que os modelos tradicionais de *machine learning* não são propriamente eficientes com esta quantidade de dados. Isso já não se aplica a modelos de *deep learning* e por isso decidimos reaproveita-las para o treino e testes destes modelos. Devemos salientar que as variáveis apresentadas são todas categóricas pelo que com a codificação (*one-hot-encoding*) cada uma gera um número de atributos igual ao número de valores diferentes, levando à geração de bastantes atributos.

Para reconsiderar estas variáveis era necessário o tratamento das mesmas, seguindo este uma abordagem semelhante à da primeira fase, que resumiremos de seguida.

- Verificação de valores nulos
- Substituição de valores nulos utilizando a seguinte técnica: verificamos a frequência de cada valor da coluna e caso esta seja superior a 94%, substituímos os NAs pelo valor modal. Caso contrário, prevemos o valor dos NAs.
- Podiamos também analisar a possibilidade um agrupamento por classes
- Após os passos anteriores era realizado o *one-hot-encoding* de cada variável

Depois deste tratamento dos dados o objetivo seria a utilização do dataset inteiro e a utilização do dataset após a aplicação do *Variance Threshold*. 
Não foi possível realizar este objetivo por não dispormos de equipamento com recursos (memória,cpu) capazes de suportar um conjunto de dados com aquela dimensão. 

## Otimização de hiperparametros

Nesta segunda fase decidimos apenas utilizar redes neuronais *feedforward*.  As redes *convolucionais* são mais adequadas a processamento de imagens, enquanto que as redes *recorrentes* são mais indicadas no processamento de sequencias, séries temporais,  texto e audio. Assim sendo o nosso objetivo foi focar-nos nas redes *feedforward* e na otimização dos seus hiperparametros. 

Os hiperparametros considerados foram os seguintes:
- Topologia da rede
- Early Stop
- Dropout
- Regularizers
- Batch size
- Epochs
- Funções da ativação
- Otimizadores

Após alguma pesquisa nesta área estes foram os parametros que consideramos como os mais revelantes para o objetivo de aumentar a performance da rede. O mais importante é obviamente a topologia da rede e é este que possivelmente provoca alterações mais drásticas no comportamento do modelo. Seguem-se os algoritmos de treino e parâmetros de regularização(utilizados para combater o *overfitting*). Os seguintes são menos relevantes mas decidimos otimiza-los mesmo assim, dando-lhe menos importância.
Tanto nos algoritmos de treino como nos otimizadores existem outros parâmetros que poderiam ser otimizados, no entanto, isso aumentaria ainda mais o espaço de procura pelo que decidimos não seguir essa abordagem.

A otimização de hiperparâmetros foi realizada utilizando uma procura aleatória de N redes diferentes, sendo que definimos o N como quinze, para não potencializar o *bias* nos dados de validação.

Aplicamos esta metedologia tanto ao *dataset* total como aos dados obtidos após a aplicação do filtro de variância.



## Optimização de hyper-parâmetros

A ideia é que esta função selecione um número X de diferentes redes.

Depois vamos criar uma função que para cada rede, vai fazer o treino e testar o score com os dados de validação. Essa função depois ordena consoante o melhor valor de AUC.

In [1]:
from sklearn.model_selection import ParameterSampler
import random as r
def selecaoHyperParametros(d,neuronios,nrCamadas,reg,n_it):
    lista_parametros = list(ParameterSampler(d, n_iter=n_it, random_state=10))
    r.seed(10)
    i = 0
    for var in lista_parametros:
        var['nrCamadas'] = r.choice(nrCamadas)
        
        var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
            
        var['regularizer'] = r.choice([0,1,2,3])
        aux = var['regularizer']
        if aux == 1:
            var['l1'] = r.choice(reg)
        elif aux == 2:
            var['l2'] = r.choice(reg)
        elif aux == 3:
            var['l1'] = r.choice(reg)
            var['l2'] = r.choice(reg)
        else:
            pass
        
        while var in lista_parametros[:i]:
            var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
            aux = var['regularizer']
            if aux == 1:
                var['l1'] = r.choice(reg)
            elif aux == 2:
                var['l2'] = r.choice(reg)
            elif aux == 3:
                var['l1'] = r.choice(reg)
                var['l2'] = r.choice(reg)
            else:
                pass
        i+=1
    return lista_parametros

Função que cria uma rede consoante os parametros passados

In [2]:
from keras import models,layers,regularizers
def criaRede(param,inputSize):
    model=models.Sequential()
    aux = param['regularizer']
    kernel_reg = None
    if aux == 1:
        kernel_reg = regularizers.l1(param['l1'])
    elif aux == 2:
        kernel_reg = regularizers.l2(param['l2'])
    elif aux == 3:
        kernel_reg = regularizers.l1_l2(l1=param['l1'],l2=param['l2'])
    else:
        pass
    
    model.add(layers.Dense(param['topologia'][0],activation=param['ativacao'],
                           kernel_regularizer=kernel_reg,input_shape=(inputSize,)))
    if param['dropout'] > 0:
        model.add(layers.Dropout(param['dropout']))
    for var in param['topologia'][1:]:
        model.add(layers.Dense(var,activation=param['ativacao'],kernel_regularizer=kernel_reg))
        if param['dropout'] > 0:
            model.add(layers.Dropout(param['dropout']))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer=param['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return model

Using TensorFlow backend.


Função que cria uma lista de diferentes parametros a testar.

Para cada um dos elementos da lista (parametros) cria uma rede e realiza o treino da mesma, calculando de seguida o *score*.

Guarda todas as configurações juntamente com o *score* e retorna-as.

In [3]:
from sklearn.metrics import roc_curve, auc
from keras.callbacks import EarlyStopping
def optimizacaoHyperParametros(d,neuronios,nrCamadas,reg,trainX,trainY,valX,valY,n_it):
    params = selecaoHyperParametros(d,neuronios,nrCamadas,reg,n_it)
    for param in params:
        rede = criaRede(param,trainX.shape[1])
        if param['early_stopping'] > 0:
            early = EarlyStopping(monitor='val_loss', patience=param['early_stopping'],
                                  min_delta=0, verbose=True, mode='auto')
            callb = [early]
            history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY),
                                callbacks=callb,
                              verbose=False)
        else:
            history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY),
                              verbose=False)
        pred = rede.predict(valX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(valY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        param['score'] = score
        print(param)
    return params

In [4]:
dicionario = {
    'ativacao':['relu','tanh','sigmoid','linear'],
    'epochs':[10,20],
    'batch_size':[64,128,256,512],
    'optimizer':['rmsprop','adam','sgd'],
    'dropout':[0.0,0.1,0.2,0.3,0.4],
    'confs':[0,1,2,3],
    'early_stopping':[0,4,5]
}
neuronios = [2,3,4,5,8,10,16,32,64]
valores_l1 = [0.1,0.01,0.001] 
nrCamadas = [1,2,3,4,5,6,7,8]

Carregamento dos dados

In [5]:
import pandas as pd
numericos = ['AVProductsInstalled',
'AVProductsEnabled',
'Census_ProcessorCoreCount',
'Census_PrimaryDiskTotalCapacity',
'Census_SystemVolumeTotalCapacity',
'Census_TotalPhysicalRAM',
'Census_InternalPrimaryDiagonalDisplaySizeInInches',
'Census_InternalPrimaryDisplayResolutionHorizontal',
'Census_InternalPrimaryDisplayResolutionVertical',
'Census_InternalBatteryNumberOfCharges']
dtype = {}
for df in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=10):
    for var in df.columns:
        if var not in numericos:
            dtype[var] = 'int8'
    break

In [6]:
import gc
del df
gc.collect()

0

In [25]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=50000,dtype=dtype):
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12


In [8]:
trainX = auxPred.loc[:499999,auxPred.columns!='HasDetections']
valX = auxPred.loc[500000:549999,auxPred.columns!='HasDetections']
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']

In [9]:
del auxPred
gc.collect()

14

In [10]:
trainX.shape

(500000, 707)

In [11]:
valX.shape

(50000, 707)

In [12]:
trainY.shape

(500000,)

In [13]:
valY.shape

(50000,)

Filtro de variância

In [14]:
import gc
import math
from sklearn.feature_selection import VarianceThreshold
def realizaVarThreshold():
    indices = []
    col = trainX.columns
    total = len(col)
    chunk = math.floor(total / 10)
    print(chunk)
    quantos = 0
    for i in range(chunk):
        sel = VarianceThreshold(threshold=0.001)
        try:
            sel.fit(trainX[col[quantos:quantos+10]])
            aux = [i+quantos for i in sel.get_support(indices=True)]
            indices.extend(aux)
        except:
            pass
        del sel
        gc.collect()
        quantos = quantos+10
    sel = VarianceThreshold(threshold=0.001)
    try:
        sel.fit(trainX[col[quantos:quantos+7]])
        indices.extend(sel.get_support(indices=True))
    except:
        pass
    del sel
    gc.collect()
    return indices

In [15]:
indices = realizaVarThreshold()
print(len(indices))
col = []
coln = trainX.columns
for i in indices:
       col.append(coln[i])

70
300


### Optimização de hiper-parâmetros para os dados com o filtro de variância

In [16]:
res = optimizacaoHyperParametros(dicionario,neuronios,nrCamadas,valores_l1,
                          trainX[col],trainY,
                          valX[col],valY,
                                15)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 00006: early stopping
{'optimizer': 'sgd', 'epochs': 20, 'early_stopping': 4, 'dropout': 0.1, 'confs': 2, 'batch_size': 512, 'ativacao': 'relu', 'nrCamadas': 1, 'topologia': [5], 'regularizer': 0, 'score': 0.5}
Epoch 00006: early stopping
{'optimizer': 'rmsprop', 'epochs': 20, 'early_stopping': 5, 'dropout': 0.1, 'confs': 3, 'batch_size': 64, 'ativacao': 'linear', 'nrCamadas': 4, 'topologia': [8, 8, 4, 32], 'regularizer': 0, 'score': 0.5088344840850655}
Epoch 00010: early stopping
{'optimizer': 'rmsprop', 'epochs': 10, 'early_stopping': 5, 'dropout': 0.4, 'confs': 2, 'batch_size': 512, 'ativacao': 'relu', 'nrCamadas': 8, 'topologia': [4, 4, 64, 64, 2, 32, 10, 5], 'regularizer': 2, 'l2': 0.001, 'score': 0.5}
{'optimizer': 'rmsprop', 'epochs': 20, 'ea

In [17]:
resultado = pd.DataFrame(res)
resultado.sort_values(by=['score'],ascending=False)

,ativacao,batch_size,confs,dropout,early_stopping,epochs,l1,l2,nrCamadas,optimizer,regularizer,score,topologia
9,tanh,128,2,0.3,4,10,0.10,0.100,3,sgd,3,0.528498,"[16, 8, 5]"
1,linear,64,3,0.1,5,20,NaN,NaN,4,rmsprop,0,0.508834,"[8, 8, 4, 32]"
13,tanh,512,0,0.1,5,10,NaN,NaN,7,rmsprop,0,0.508821,"[8, 5, 5, 16, 10, 2, 64]"
0,relu,512,2,0.1,4,20,NaN,NaN,1,sgd,0,0.500000,[5]
2,relu,512,2,0.4,5,10,NaN,0.001,8,rmsprop,2,0.500000,"[4, 4, 64, 64, 2, 32, 10, 5]"
3,sigmoid,128,2,0.1,0,20,0.01,NaN,5,rmsprop,1,0.500000,"[8, 16, 10, 3, 16]"
4,relu,512,1,0.0,4,20,NaN,0.001,7,adam,2,0.500000,"[2, 2, 3, 64, 4, 5, 64]"
5,tanh,128,2,0.1,4,20,0.01,0.001,8,sgd,3,0.500000,"[5, 2, 10, 32, 3, 4, 5, 2]"
6,linear,128,1,0.3,0,20,0.01,NaN,2,sgd,1,0.500000,"[8, 32]"
7,tanh,64,0,0.4,4,10,0.01,NaN,6,sgd,1,0.500000,"[64, 3, 64, 16, 5, 64]"


In [18]:
(trainY.memory_usage() + valY.memory_usage() + 
 trainX.memory_usage().sum() + valX.memory_usage().sum()) / (1000*1000)

427.900328

### Optimização de hiper-parâmetros para os dados totais

In [19]:
res = optimizacaoHyperParametros(dicionario,neuronios,nrCamadas,valores_l1,
                          trainX[:300000],trainY[:300000],
                          valX,valY,
                                15)

Epoch 00005: early stopping
{'optimizer': 'sgd', 'epochs': 20, 'early_stopping': 4, 'dropout': 0.1, 'confs': 2, 'batch_size': 512, 'ativacao': 'relu', 'nrCamadas': 1, 'topologia': [5], 'regularizer': 0, 'score': 0.5}
Epoch 00007: early stopping
{'optimizer': 'rmsprop', 'epochs': 20, 'early_stopping': 5, 'dropout': 0.1, 'confs': 3, 'batch_size': 64, 'ativacao': 'linear', 'nrCamadas': 4, 'topologia': [8, 8, 4, 32], 'regularizer': 0, 'score': 0.5}
Epoch 00008: early stopping
{'optimizer': 'rmsprop', 'epochs': 10, 'early_stopping': 5, 'dropout': 0.4, 'confs': 2, 'batch_size': 512, 'ativacao': 'relu', 'nrCamadas': 8, 'topologia': [4, 4, 64, 64, 2, 32, 10, 5], 'regularizer': 2, 'l2': 0.001, 'score': 0.5}
{'optimizer': 'rmsprop', 'epochs': 20, 'early_stopping': 0, 'dropout': 0.1, 'confs': 2, 'batch_size': 128, 'ativacao': 'sigmoid', 'nrCamadas': 5, 'topologia': [8, 16, 10, 3, 16], 'regularizer': 1, 'l1': 0.01, 'score': 0.5}
Epoch 00010: early stopping
{'optimizer': 'adam', 'epochs': 20, 'earl

In [20]:
resultado = pd.DataFrame(res)
resultado.sort_values(by=['score'],ascending=False)

,ativacao,batch_size,confs,dropout,early_stopping,epochs,l1,l2,nrCamadas,optimizer,regularizer,score,topologia
13,tanh,512,0,0.1,5,10,NaN,NaN,7,rmsprop,0,0.527676,"[8, 5, 5, 16, 10, 2, 64]"
14,linear,512,2,0.3,5,10,NaN,NaN,1,sgd,0,0.508834,[5]
0,relu,512,2,0.1,4,20,NaN,NaN,1,sgd,0,0.500000,[5]
1,linear,64,3,0.1,5,20,NaN,NaN,4,rmsprop,0,0.500000,"[8, 8, 4, 32]"
2,relu,512,2,0.4,5,10,NaN,0.001,8,rmsprop,2,0.500000,"[4, 4, 64, 64, 2, 32, 10, 5]"
3,sigmoid,128,2,0.1,0,20,0.01,NaN,5,rmsprop,1,0.500000,"[8, 16, 10, 3, 16]"
5,tanh,128,2,0.1,4,20,0.01,0.001,8,sgd,3,0.500000,"[5, 2, 10, 32, 3, 4, 5, 2]"
6,linear,128,1,0.3,0,20,0.01,NaN,2,sgd,1,0.500000,"[8, 32]"
7,tanh,64,0,0.4,4,10,0.01,NaN,6,sgd,1,0.500000,"[64, 3, 64, 16, 5, 64]"
8,tanh,128,1,0.3,0,20,NaN,0.100,7,sgd,2,0.500000,"[64, 8, 16, 3, 8, 32, 4]"


### Seleção do melhor modelo
Como podemos ver nas tabelas acima o melhor modelo nos dados de validação foi o modelo que contém as seguintes caraterísticas:
- topologia: [16, 8, 5]
- função de ativação: tanh
- batch_size: 128
- dropout: 0.3
-early_stopping: 4
- epochs: 10
- regularizers:
    - l1: 0.1
    - l2: 0.1
- optimizer: sgd (Gradiente descendente estocástico)

Para além disso este modelo usa apenas as colunas que passam no filtro de variância.

Este modelo obteve um score de 0.528498 (AUC).

Carregar os dados para testar o erro nos dados de teste

In [30]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=50000,dtype=dtype):
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12


In [31]:
trainX = auxPred.loc[:499999,auxPred.columns!='HasDetections']
valX = auxPred.loc[500000:549999,auxPred.columns!='HasDetections']
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']
testX = auxPred.loc[550000:599999,auxPred.columns!='HasDetections']
testY = auxPred.loc[550000:599999,'HasDetections']

In [32]:
del auxPred
gc.collect()

171

In [38]:
params = {'optimizer': 'sgd', 'epochs': 10, 'early_stopping': 4, 
          'dropout': 0.3, 'confs': 2, 'batch_size': 128, 'ativacao': 'tanh', 'nrCamadas': 3, 
          'topologia': [16, 8, 5], 'regularizer': 3, 'l1': 0.1, 'l2': 0.1}
rede = criaRede(params,trainX[col].shape[1])

Criar a rede com a melhor configuração e treinar a mesma da mesma forma que foi realizada anteriormente.

Depois é verificado o score nos dados de teste.

In [39]:
if params['early_stopping'] > 0:
    early = EarlyStopping(monitor='val_loss', patience=params['early_stopping'],
                          min_delta=0, verbose=True, mode='auto')
    callb = [early]
    history = rede.fit(trainX[col],
                        trainY,
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        validation_data=(valX[col],valY),
                        callbacks=callb)
else:
    history = rede.fit(trainX[col],
                        trainY,
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        validation_data=(valX[col],valY))
pred = rede.predict(testX[col])
false_positive_rate, true_positive_rate, thresholds = roc_curve(testY, pred)
score = auc(false_positive_rate, true_positive_rate)

Train on 500000 samples, validate on 50000 samples
Epoch 1/10
500000/500000 [==============================] - 19s 39us/step - loss: 1.5895 - acc: 0.5017 - val_loss: 0.9419 - val_acc: 0.4997
Epoch 2/10
500000/500000 [==============================] - 16s 33us/step - loss: 0.9419 - acc: 0.4999 - val_loss: 0.9420 - val_acc: 0.4997
Epoch 3/10
500000/500000 [==============================] - 16s 33us/step - loss: 0.9419 - acc: 0.4983 - val_loss: 0.9420 - val_acc: 0.4997
Epoch 4/10
500000/500000 [==============================] - 17s 33us/step - loss: 0.9419 - acc: 0.5002 - val_loss: 0.9419 - val_acc: 0.4997
Epoch 5/10
500000/500000 [==============================] - 16s 33us/step - loss: 0.9419 - acc: 0.4993 - val_loss: 0.9420 - val_acc: 0.5003
Epoch 6/10
500000/500000 [==============================] - 16s 33us/step - loss: 0.9419 - acc: 0.4990 - val_loss: 0.9419 - val_acc: 0.5003
Epoch 7/10
500000/500000 [==============================] - 16s 33us/step - loss: 0.9419 - acc: 0.4998 - val_

Score final do modelo

In [40]:
score

0.5010305127845165

# Conclusão

Como foi referido anteriormente o objetivo da segunda fase passava por aplicar métodos de aprendizagem máquina *deep* e comparar o resultado obtido com o resultada da fase transata. Nesta fase o foco recaiu sobre as redes neuronais sendo que o aspeto que foi intensamente abordado foi a otimização dos hiper-parametros dessas mesmas redes. Através dessa otimização foi-nos possível criar e testar uma quantidade muito significativa de redes diferentes, com caracteristicas bastantes diversas. 

Como pudemos verificar os resultados obtidos não foram os esperados, visto serem inferiores aos apurados com os métodos tradicionais, sendo que essa diferença é bastante considerável tendo um **score** de 0.5010305127845165 neste modelo quando tinha sido de 0.6634414889076333 para o melhor modelo na fase anterior.

Aquilo que poderiamos ter realizado e que poderia potencializar um maior acerto, foi como já referimos, a utilização de mais atributos ou até mesmo a utilização de um diferente tipo de rede. Mesmo assim não achamos que os resultados iriam melhorar de forma significativa com a adoção destas medidas. 

Um problema que o grupo não conseguiu resolver foi que a métrica usada para monitorizar o *EarlyStopping* foi a loss nos casos de validação, no entanto esta métrica não é a utilizada para avaliar o modelo. Idealmente seria usado o **AUC** como monitor, no entanto não conseguimos aplicar o cálculo do mesmo em *batches*, sendo algo que pode ser melhorado e pode melhorar os resultados.

No final podemos concluir que o melhor modelo foi o obtido na primeira fase o que nos leva a perceber que apesar destes novos métodos parecerem mais robustos quando comparados com os métodos mais tradicionais, isso não é uma verdade absoluta. O que podemos perceber é que cada método pode ser mais apropriado para um determinado caso, não existindo um melhor ou que se comporte aproximadamente bem em todos os casos. No nosso exemplo o mais apropriado é um método de *machine learning* tradicional, as **Random Forests**.